In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import cv2
import time
from skimage import morphology
import seaborn as sns
import collections
import random
plt.rcParams['figure.figsize'] = (20,12)
%config InlineBackend.figure_format = 'retina'


In [72]:
def randomize_in_ROI(target_images_list, ROI_image, randomize_size=1, peripheral_shuffle=True):
    """
        Args(引数)
            target_images_list: list型, 各画像は2次元配列
            ROI_image: numpy.ndarray型, 2次元配列でbinarize画像
            randomize_size: int型, シャッフルしたいサイズの正方形の一辺の長さ
            peripheral_shuffle: bool型, randomize_sizeでシャッフルした後、サイズ的にシャッフルできなかった箇所を1pixel単位でシャッフルするかどうか
        
        Return(返り値)
            randomized_imgs_list: list型, target_imagesで入っていた画像の順でrandomizeした画像を返す
    """
    
    H = ROI_image.shape[0] // randomize_size
    W = ROI_image.shape[1] // randomize_size
    ROI_image_copy = (ROI_image>0).astype(int)
    
    # ROI_imageをrandomize_sizeを一辺とする正方形で区切り、各正方形においてROIに含まれている正方形のみを抽出する
    grid_list = list()
    for h in range(H):
        for w in range(W):
            if np.min(ROI_image_copy[h*randomize_size:(h+1)*randomize_size, w*randomize_size:(w+1)*randomize_size]) == 1:
                grid_list.append([h, w])
                
    shuffle_grid_list = random.sample(grid_list, len(grid_list))
    
    if peripheral_shuffle:
        # randomize_sizeを一辺とする正方形でのshuffleに含まれていないROIのpixelを抽出する
        for h, w in grid_list:
            ROI_image_copy[h*randomize_size:(h+1)*randomize_size, w*randomize_size:(w+1)*randomize_size] -= 1

        ROI_image_coords = ROI_image_copy.nonzero()
        pixel_list = list( [h, w] for h, w in zip(ROI_image_coords[0], ROI_image_coords[1]) )
        shuffle_pixel_list = random.sample(pixel_list, len(pixel_list))

    
    # shuffleを実行する
    randomized_imgs_list = list()
    for i, target_image in enumerate(target_images_list):
        print(f"{str(i+1)}枚目のrandom shuffleを開始します")
        randomized_img = np.zeros_like(target_image)
        for grid_i, shuffle_grid_i in zip(grid_list, shuffle_grid_list):
            randomized_img[
                shuffle_grid_i[0]*randomize_size:(shuffle_grid_i[0]+1)*randomize_size, 
                shuffle_grid_i[1]*randomize_size:(shuffle_grid_i[1]+1)*randomize_size
            ] = target_image[
                grid_i[0]*randomize_size:(grid_i[0]+1)*randomize_size, 
                grid_i[1]*randomize_size:(grid_i[1]+1)*randomize_size
            ]
        if peripheral_shuffle:
            for pixel_i, shuffle_pixel_i in zip(pixel_list, shuffle_pixel_list):
                randomized_img[shuffle_pixel_i[0], shuffle_pixel_i[1]] = target_image[pixel_i[0], pixel_i[1]]
        print(f"{str(i+1)}枚目のrandom shuffleが終わりました")
        randomized_imgs_list.append(randomized_img)

        
    return randomized_imgs_list

In [154]:
threshold_O = 120
for i in range(1,4):
    file_name = r"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20  - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    Mito = cv2.imread(file_name + str(i) + ".tif_OxPHOS.tif",cv2.IMREAD_ANYDEPTH)
    FKBP8 = cv2.imread(file_name + str(i) + ".tif_FKBP8.tif",cv2.IMREAD_ANYDEPTH)    
        
    ret_M ,th_M = cv2.threshold(Mito,threshold_O,65535,cv2.THRESH_BINARY)
    th_M = th_M/65535
    
    # Converting to 8bit image
    th_M = (th_M).astype('uint8')
        
    randomized_imgs_list = randomize_in_ROI(
    target_images_list=[FKBP8], 
    ROI_image=th_M, 
    randomize_size=1, 
    peripheral_shuffle=True) 
    suffuled_FKBP8_on_mito = randomized_imgs_list[0]
    cv2.imwrite(file_name + str(i) + "_FKBP8_on_mito_randomized.tif", suffuled_FKBP8_on_mito)

1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました


In [155]:
def main2(file_name, threshold_P, threshold_O, threshold_F):
    
    PDZD8_with_FKBP8_on_mito_list = []
    PDZD8_with_rFKBP8_on_mito_list = []
    PDZD8_on_mito_list = []

    for i in range(1,4):
        PDZD8 =  cv2.imread(file_name + str(i) + ".tif_PDZD8.tif",cv2.IMREAD_ANYDEPTH)
        Mito = cv2.imread(file_name + str(i) + ".tif_OxPHOS.tif",cv2.IMREAD_ANYDEPTH)
        FKBP8 = cv2.imread(file_name + str(i) + ".tif_FKBP8.tif",cv2.IMREAD_ANYDEPTH)
        rFKBP8_on_mito = cv2.imread(file_name + str(i) + "_FKBP8_on_mito_randomized.tif",cv2.IMREAD_ANYDEPTH)
        
        ret_M ,th_M = cv2.threshold(Mito,threshold_O,65535,cv2.THRESH_BINARY)
        th_M = th_M/65535

        ret_F ,th_F = cv2.threshold(FKBP8,threshold_F,65535,cv2.THRESH_BINARY)
        th_F = th_F/65535
        FKBP8 = FKBP8 - threshold_F
        FKBP8 = FKBP8 * th_F
        
        ret_rF ,th_rF = cv2.threshold(rFKBP8_on_mito,threshold_F,65535,cv2.THRESH_BINARY)
        th_rF = th_rF/65535
        rFKBP8_on_mito = rFKBP8_on_mito - threshold_F
        rFKBP8_on_mito = rFKBP8_on_mito * th_rF 
    
        ret_P ,th_P = cv2.threshold(PDZD8,threshold_P,65535,cv2.THRESH_BINARY)
        th_P = th_P/65535
        PDZD8 = PDZD8 - threshold_P
        PDZD8 = PDZD8 * th_P
    
        # Converting to 8bit image
        th_P = (th_P).astype('uint8')
        th_F = (th_F).astype('uint8')
        th_M = (th_M).astype('uint8')
        th_rF = (th_rF).astype('uint8')
        
        PDZD8_with_FKBP8_on_mito = PDZD8*th_F*th_M
        PDZD8_with_rFKBP8_on_mito = PDZD8*th_rF*th_M
        PDZD8_on_mito = PDZD8*th_M
        
        PDZD8_with_FKBP8_on_mito_list = np.append(PDZD8_with_FKBP8_on_mito_list, PDZD8_with_FKBP8_on_mito)
        PDZD8_with_rFKBP8_on_mito_list = np.append(PDZD8_with_rFKBP8_on_mito_list, PDZD8_with_rFKBP8_on_mito)
        PDZD8_on_mito_list = np.append(PDZD8_on_mito_list, PDZD8_on_mito)
        
    Mander_of_mito_PDZD8  = np.sum(PDZD8_with_FKBP8_on_mito_list)/np.sum(PDZD8_on_mito_list)
    r_Mander_of_mito_PDZD8  = np.sum(PDZD8_with_rFKBP8_on_mito_list)/np.sum(PDZD8_on_mito_list)
    
    ans = [Mander_of_mito_PDZD8, r_Mander_of_mito_PDZD8]
    return ans

In [156]:
df2_1 = pd.DataFrame(data=None, index=None, columns= ["Mander_of_mito_PDZD8", "r_Mander_of_mito_PDZD8"], dtype=None, copy=False)
for i in [1,6,7,9,10]:
    file_name = r"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 " + str(i) + " - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    threshold_P = 50
    threshold_O = 120
    threshold_F = 300 
    df2_1.loc[file_name] = main2(file_name, threshold_P, threshold_O, threshold_F)

In [157]:
df2_2 = pd.DataFrame(data=None, index=None, columns= ["Mander_of_mito_PDZD8", "r_Mander_of_mito_PDZD8"], dtype=None, copy=False)
for i in [1,2,4,5]:
    file_path = r"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 " + str(i) + " - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    threshold_P = 40
    threshold_O = 120
    threshold_F = 400
    df2_2.loc[file_path] = main2(file_path,threshold_P,threshold_O,threshold_F)

In [158]:
df2 = pd.concat([df2_1, df2_2])
df2

,Mander_of_mito_PDZD8,r_Mander_of_mito_PDZD8
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 1 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.492669,0.419416
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 6 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.657555,0.374029
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 7 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.105009,0.120424
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 9 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.328838,0.301599
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 10 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.174836,0.123653
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 1 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.460813,0.358909
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 2 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.383390,0.264240
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 4 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.130766,0.094042
"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 5 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.428338,0.341532


In [159]:
df2.to_csv("PDZD8_with_FKBP8_on_mito_mander.csv")